<a href="https://colab.research.google.com/github/masfworld/sparkstreaming_datahack_webinar/blob/master/SparkStreaming_Datahack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Spark Streaming


<a href="https://datahack.es">
  <img src="https://www.datahack.es/wp-content/uploads/2019/10/Todo_vertical.png" Big Data Spain"" width="40%">
  
  
**Author**:
miguel.sotomayor@sidesna.es\
https://www.linkedin.com/in/miguelsotomayorf/ \
https://github.com/masfworld





# Prerrequisites

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install py4j

     |████████████████████████████████| 204kB 2.7MB/s 


In [0]:
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/libs/libs-kafka.zip --directory-prefix=/content/spark-2.4.5-bin-hadoop2.7/jars/
!unzip -n /content/spark-2.4.5-bin-hadoop2.7/jars/libs-kafka.zip -d /content/spark-2.4.5-bin-hadoop2.7/jars/
!ls /content/spark-2.4.5-bin-hadoop2.7/jars/*kafka*

Archive:  /content/spark-2.4.5-bin-hadoop2.7/jars/libs-kafka.zip
  inflating: /content/spark-2.4.5-bin-hadoop2.7/jars/kafka-clients-2.0.0.jar  
  inflating: /content/spark-2.4.5-bin-hadoop2.7/jars/lz4-java-1.4.1.jar  
  inflating: /content/spark-2.4.5-bin-hadoop2.7/jars/slf4j-api-1.7.25.jar  
  inflating: /content/spark-2.4.5-bin-hadoop2.7/jars/snappy-java-1.1.7.1.jar  
  inflating: /content/spark-2.4.5-bin-hadoop2.7/jars/spark-sql-kafka-0-10_2.11-2.4.5.jar  
  inflating: /content/spark-2.4.5-bin-hadoop2.7/jars/unused-1.0.0.jar  
/content/spark-2.4.5-bin-hadoop2.7/jars/kafka-clients-2.0.0.jar
/content/spark-2.4.5-bin-hadoop2.7/jars/libs-kafka.zip
/content/spark-2.4.5-bin-hadoop2.7/jars/spark-sql-kafka-0-10_2.11-2.4.5.jar


Define the environment (Java & Spark homes)

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
os.environ["PYSPARK_SUBMIT_ARGS"] = ""

Starting Spark Session and print the version

In [0]:
import findspark
findspark.add_packages(["org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5"])
findspark.add_jars(["/content/spark-2.4.5-bin-hadoop2.7/jars/kafka-clients-2.0.0.jar","/content/spark-2.4.5-bin-hadoop2.7/jars/lz4-java-1.4.1-jar","/content/spark-2.4.5-bin-hadoop2.7/jars/scala-library-2.11.12.jar","/content/spark-2.4.5-bin-hadoop2.7/jars/slf4j-api-1.7.25.jar","/content/spark-2.4.5-bin-hadoop2.7/jars/snappy-java-1.1.7.1.jar","/content/spark-2.4.5-bin-hadoop2.7/jars/spark-sql-kafka-0-10_2.11-2.4.5.jar","/content/spark-2.4.5-bin-hadoop2.7/jars/spark-tags_2.11-2.4.5.jar","/content/spark-2.4.5-bin-hadoop2.7/jars/unused-1.0.0.jar"])
findspark.init("spark-2.4.5-bin-hadoop2.7")# SPARK_HOME

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.version

'2.4.5'

# Streaming from a directory

In [0]:
!head /content/sample_data/california_housing_train.csv

In [0]:
!mkdir -p /content/california_housing
!cp /content/sample_data/california_housing_train.csv /content/california_housing/
!ls /content/california_housing/

In [0]:
from pyspark.sql.types import StructType

# Read all the csv files written atomically in a directory
housingSchema = StructType()\
.add("longitude", "double")\
.add("latitude", "double")\
.add("housing_median_age", "double")\
.add("total_rooms", "double")\
.add("total_bedrooms", "double")\
.add("population", "double")\
.add("households", "double")\
.add("median_income", "double")\
.add("median_house_value", "double")

housing_df = spark \
    .readStream \
    .option("sep", ",") \
    .option("header", "true") \
    .schema(housingSchema) \
    .csv("/content/california_housing/")  


In [0]:
from pyspark.sql.functions import col, avg

housing_avg = housing_df.agg(avg(col('total_bedrooms')))

In [0]:
housing_avg \
  .writeStream \
  .outputMode("complete") \
  .format("memory") \
  .queryName("housing_avg") \
  .start()

In [0]:
spark.sql("select * from housing_avg").show()

In [0]:
!cp /content/sample_data/california_housing_test.csv /content/california_housing/

In [0]:
spark.sql("select * from housing_avg").show()

# Streaming From Kafka

In [0]:
kafka_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "host1:port1,host2:port2") \
  .option("subscribe", "topic1") \
  .load()